
# 我们的任务

垃圾邮件检测是当今互联网中机器学习的主要应用之一。几乎所有主要的电子邮件服务提供商都内置了垃圾邮件检测系统，并将这些邮件自动分类为“垃圾邮件”。

在此任务中，我们将使用朴素贝叶斯算法来创建一个模型，该模型可根据我们对该模型的训练，将数据集SMS消息分类为垃圾邮件或非垃圾邮件。对垃圾短信的外观要有一定的直觉，这一点很重要。通常这些文字中有“免费”，“赢”，“赢家”，“现金”，“奖品”之类的词，因为这些文字的目的是引起您的注意，并在某种意义上诱使您打开它们。同样，垃圾邮件消息中的单词通常用大写字母表示，并且往往使用很多感叹号。对于收件人来说，识别垃圾邮件通常很简单，而我们的目标是训练一个模型来为我们做到这一点！

能够识别垃圾邮件是一个二进制分类问题，因为邮件被分类为“垃圾邮件”或“非垃圾邮件”，别无其他。同样，这是一个监督学习问题，因为我们将向其学习的标记数据集输入模型中，以进行未来的预测。

# 总览

该项目已细分为以下步骤：

- 步骤0：朴素贝叶斯定理简介
- 步骤1.1：了解我们的数据集
- 步骤1.2：数据预处理
- 步骤2.1：单词袋（BoW）
- 步骤2.2：从头开始实施BoW
- 步骤2.3：在scikit-learn中实现单词袋
- 步骤3.1：培训和测试集
- 步骤3.2：将词袋处理应用于我们的数据集。
- 步骤4.1：从头开始实施贝叶斯定理
- 步骤4.2：从零开始实施朴素贝叶斯
- 步骤5：使用scikit-learn的朴素贝叶斯实现
- 步骤6：评估我们的模型
- 步骤7：结论


### 步骤0：朴素贝叶斯定理简介

贝叶斯定理（Bayes theorem）是由贝弗斯牧师（Reverend Bayes）开发的最早的概率推断算法之一（他曾经尝试尝试推断上帝的存在），并且在某些用例中仍然表现出色。

最好通过一个例子来理解这个定理。假设您是特勤局的成员，并且在他/她的一次竞选演讲中被部署来保护民主党总统候选人。作为一个对所有人开放的公共活动，您的工作并不容易，而且您必须时刻警惕威胁。因此，一个起点就是为每个人设置一定的威胁因素。因此，根据个人的特征（例如年龄，性别）和其他较小的因素（例如，提包的人？），这个人看起来紧张吗？等等。您可以就此人是否可行威胁做出判断。

如果某人在所有方框中打勾，直到其超过您的怀疑阈值，您就可以采取行动，将该人从附近移开。贝叶斯定理的工作原理与我们基于某些相关事件的概率（年龄，性别，是否有行李，人的紧张程度等）计算事件（某人是威胁）的概率相同。

要考虑的一件事是这些功能之间的独立性。例如，如果一个孩子对事件感到紧张，那么那个人受到威胁的可能性就不如说是一个长大的人感到紧张。为了进一步说明这一点，我们考虑了以下两个特征：年龄和紧张感。假设我们单独查看这些功能，我们可以设计一个模型，将所有紧张的人标记为潜在威胁。但是，我们很可能会产生很多误报，因为参加活动的未成年人很有可能会感到紧张。因此，通过考虑一个人的年龄以及“神经质”功能，我们肯定会获得关于谁是潜在威胁，谁不是潜在威胁的更准确的结果。

这是定理的“天真”位，它认为每个特征彼此独立，但并非总是如此，因此会影响最终判断。

简而言之，贝叶斯定理根据某些其他事件的联合概率分布（在我们的情况下，消息中某些单词的出现）来计算发生某事件（在我们的情况下，邮件为垃圾邮件）的概率。在任务的稍后部分，我们将深入探讨贝叶斯定理的工作原理，但首先，让我们了解将要使用的数据。

### 步骤1.1：了解我们的数据集


我们将使用UCI机器学习存储库中的[数据集](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)，该库具有非常好的数据集，可用于实验研究。 直接数据链接是[here](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/)。

Here's a preview of the data:

<img src="images/dqnb.png" height="1242" width="1242">

数据集中的列当前未命名，如您所见，共有2列。

第一列采用两个值，“ ham”表示邮件不是垃圾邮件，“ spam”表示邮件是垃圾邮件。

第二列是正在分类的SMS消息的文本内容。

>  说明：
* 使用read_table方法将数据集导入到熊猫数据框。 由于这是一个制表符分隔的数据集，因此我们将使用'\t'作为指定此格式的'sep'参数的值。
* 另外，通过在`read_table()`的`names`参数中指定一个列表['label，'sms_message']来重命名列名称。
* 使用新的列名打印数据框的前五个值。

In [1]:
'''
Solution
'''
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table('smsspamcollection/SMSSpamCollection',
                   sep='\t',
                   header=None, 
                   names=['label', 'sms_message'])

# Output printing out first 5 rows
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### 步骤1.2：数据预处理 ###

现在我们对数据集的外观有了基本的了解，让我们将标签转换为二进制变量，为便于计算，将0表示`ham`（即不是垃圾邮件），将1表示`spam`。

您可能想知道为什么我们需要执行此步骤？ 答案在于scikit-learn如何处理输入。 Scikit-learn仅处理数字值，因此，如果我们将标签值保留为字符串，则scikit-learn将在内部进行转换（更具体地说，字符串标签将转换为未知的float值）。

如果我们将标签保留为字符串，我们的模型仍然可以做出预测，但是稍后在计算性能指标时（例如，在计算精度和召回率时）可能会出现问题。 因此，为避免以后发生意外的“陷阱”，优良作法是将分类值作为整数输入到我们的模型中。

> 说明:
* 使用映射方法，将'label'列中的值转换为数字值，如下所示：`{'ham':0, 'spam':1}`它将`ham`值映射为0，将`spam`值映射为1。
* 另外，要了解我们正在处理的数据集的大小，请使用“shape”打印出行数和列数。

In [2]:
'''
Solution
'''
df['label'] = df.label.map({'ham': 0, 'spam': 1})
print(df.shape)
df.head() # returns (rows, columns)

(5572, 2)


,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### 步骤2.1：单词袋 ###

我们的数据集中有大量文本数据（5,572行数据）。 大多数机器学习算法都依赖于数字数据作为输入，而电子邮件/短信通常是大量文本。

在这里，我们要介绍“词袋”（BoW）概念，该术语用于指定具有“词袋”或需要处理的文本数据集合的问题。 BoW的基本思想是获取一段文本并计算该文本中单词的出现频率。 重要的是要注意，BoW概念单独对待每个单词，单词出现的顺序无关紧要。

使用我们现在将要经历的过程，我们可以将文档集合转换为矩阵，每个文档为一行，每个单词（令牌）为列，相应的（行，列）值为频率 该文档中每个单词或标记的出现。

例如：

可以说我们有4个文档，如下所示：

`['Hello, how are you!',
'Win money, win from home.',
'Call me now',
'Hello, Call you tomorrow?']`

我们的目标是将这组文本转换为频率分布矩阵，如下所示：

<img src="images/countvectorizer.png" height="542" width="542">

正如我们所看到的，文档在行中编号，每个单词是一个列名，相应的值是该单词在文档中的出现频率。

让我们分解一下，看看我们如何使用一小组文档进行转换。

为了解决这个问题，我们将使用sklearns
[count vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) method 

它执行以下操作：

* 它对字符串进行标记化（将字符串分成单个单词），并为每个标记赋予一个整数ID。
* 计算每个标记的出现次数。

请注意：

* CountVectorizer方法自动将所有标记化的单词转换为小写形式，这样就不会像对待“ He”和“ he”那样区别对待。它使用“ lowercase”参数执行此操作，该参数默认设置为“ True”。
* 它还会忽略所有标点符号，以使后面带有标点符号的单词（例如：“ hello！”）与未带标点符号前缀或后缀的相同单词（例如：“ hello”）的对待方式不同。它使用`token_pattern`参数执行此操作，该参数具有默认正则表达式，该正则表达式选择2个或更多字母数字字符的标记。
* 第三个要注意的参数是`stop_words`参数。停用词是指语言中最常用的词。它们包括'am'，'an'，'and'，'the'等字词。通过将此参数值设置为“ english”，CountVectorizer将自动忽略在内置文件中找到的所有字词（来自我们的输入文本） scikit-learn中的英语停用词列表。这非常有用，因为当我们尝试查找某些表示垃圾邮件的关键字时，停用词会使我们的计算产生偏差。

我们将在以后的步骤中将每种方法应用到我们的模型中，但是就目前而言，在处理文本数据时要意识到可供我们使用的此类预处理技术非常重要。

### 步骤2.2：从头开始实施单词袋 ###

在我们深入scikit-learn的“语言袋”（BoW）库为我们做复杂的工作之前，让我们先自己实现它，以便我们了解幕后发生的事情。

**步骤1：将所有字符串转换为小写形式。**

假设我们有一个文档集：

```
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
```
>说明：
* 将文档集中的所有字符串转换为小写。 将它们保存到名为“ lower_case_documents”的列表中。 您可以使用lower()方法在python中将字符串转换为小写形式。


In [3]:
'''
Solution:
'''
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


**步骤2：删除所有标点符号**

> **说明：**
从文档集中的字符串中删除所有标点符号。 将它们保存到名为
'sans_punctuation_documents'。

In [4]:
'''
Solution:
'''
sans_punctuation_documents = []
import string

for i in lower_case_documents:
    # translate() 方法根据参数table给出的表(包含 256 个字符)转换字符串的字符, 要过滤掉的字符放到 del 参数中。
    # Python maketrans() 方法用于创建字符映射的转换表，对于接受两个参数的最简单的调用方式，第一个参数是字符串，表示需要转换的字符，第二个参数也是字符串表示转换的目标。
    # string.punctuation 所有的标点字符
    sans_punctuation_documents.append(i.translate(str.maketrans('', '', string.punctuation)))
    
print(sans_punctuation_documents)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


**步骤3：标记化**

将文档集中的句子标记为令牌意味着使用定界符将一个句子拆分为单个单词。 分隔符指定我们将使用什么字符来标识单词的开头和结尾（例如，我们可以使用单个空格作为分隔符来标识文档集中的单词）。

>**说明：**
使用split（）方法标记存储在“ sans_punctuation_documents”中的字符串。 并存储最终文档集
在名为“ preprocessed_documents”的列表中。


In [5]:
'''
Solution:
'''
preprocessed_documents = []
for i in sans_punctuation_documents:
    preprocessed_documents.append(i.split(' '))
print(preprocessed_documents)

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


**步骤4：计数频率**

现在，我们已经以所需的格式设置了文档集，我们可以继续计算文档集中每个文档中每个单词的出现次数。 为此，我们将使用Python`collections`库中的`Counter`方法。

“ Counter”对列表中每个项目的出现进行计数，并返回一个字典，其中键为要计数的项目，而对应的值为列表中该项目的计数。

> **说明：**
使用Counter（）方法和preprocessed_documents作为输入，创建一个字典，其中的键是每个文档中的每个单词，而对应的值是该单词出现的频率。 将每个Counter字典另存为名为“ frequency_list”的列表中的项目。


In [6]:
'''
Solution
'''
frequency_list = []
import pprint
from collections import Counter

for i in preprocessed_documents:
    frequency_counts = Counter(i)
    frequency_list.append(frequency_counts)
    
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


恭喜你！ 您已经从头开始实施了“语言袋”流程！ 正如我们在先前的输出中看到的，我们有一个频率分布字典，可以清晰地查看正在处理的文本。

现在，我们应该对scikit-learn的`sklearn.feature_extraction.text.CountVectorizer`方法在幕后发生的事情有深刻的了解。

现在，我们将在下一步中实现`sklearn.feature_extraction.text.CountVectorizer`方法。

### 步骤2.3：在scikit-learn中实现单词袋 ###

现在我们已经从头开始实现了BoW概念，让我们继续使用scikit-learn以简洁明了的方式完成此过程。 我们将使用与上一步相同的文档集。

In [7]:
'''
Here we will look to create a frequency matrix on a smaller document set to make sure we understand how the 
document-term matrix generation happens. We have created a sample document set 'documents'.
'''
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

> **说明：**
导入sklearn.feature_extraction.text.CountVectorizer方法并创建一个名为“count_vector”的实例。

In [8]:
'''
Solution
'''
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

**使用CountVectorizer()进行数据预处理**

在步骤2.2中，我们从头开始实现了CountVectorizer()方法的一个版本，该版本需要首先清除我们的数据。清理工作涉及将我们的所有数据转换为小写并删除所有标点符号。 CountVectorizer()具有某些参数，这些参数将为我们处理这些步骤。他们是：

* `lowercase = True`
    
    小写参数的默认值为True，它将所有文本转换为小写形式。


* `token_pattern = (?u)\\b\\w\\w+\\b`
    
    `token_pattern`参数的默认正则表达式值为`(?u)\\b\\w\\w+\\b`，它将忽略所有标点符号并将其视为定界符，同时接受长度大于或等于的字母数字字符串等于2，作为单个标记或单词。
    
* `stop_words`

    如果将`stop_words`参数设置为`english`，则会从文档集中删除与scikit-learn中定义的英语停用词列表匹配的所有词。考虑到数据集的大小以及我们正在处理SMS消息而不是诸如电子邮件这样的较大文本源的事实，我们将不会设置此参数值。

您可以通过简单地如下打印对象来查看您的`count_vector`对象的所有参数值：


In [9]:
'''
Practice node:
Print the 'count_vector' object which is an instance of 'CountVectorizer()'
'''
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


> **说明：**
使文档数据集适合使用fit()创建的CountVectorizer对象，并获取单词列表
使用get_feature_names()方法将其分类为功能。

In [10]:
'''
Solution:
'''
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

`get_feature_names()`方法返回该数据集的特征名称，该特征名称是构成'documents'词汇的一组单词。

> **说明：**
创建一个矩阵，其中行是4个文档中的每个文档，列是每个单词。
相应的（行，列）值是该单词（在列中）在特定单词中出现的频率
文档（在行中）。 您可以使用transform()方法，并将文档数据集作为
论点。 transform()方法返回一个numpy整数矩阵，您可以使用以下方法将其转换为数组
toarray()。 将该数组称为“doc_array”


In [11]:
'''
Solution
'''
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

现在，我们可以根据文档中单词的频率分布来清晰地表示文档。 为了更容易理解，我们的下一步是将该数组转换为数据框并适当命名列。

> **说明：**
将获得的数组转换为“doc_array”，并转换为数据框，并将列名设置为
单词名称（您之前使用get_feature_names()计算出的名称。将数据框称为“frequency_matrix”。


In [12]:
'''
Solution
'''
frequency_matrix = pd.DataFrame(doc_array, 
                                columns = count_vector.get_feature_names())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


恭喜你！您已经为我们创建的文档数据集成功实现了“单词袋”问题。

开箱即用使用此方法可能引起的一个潜在问题是，如果我们的文本数据集非常大（例如，如果我们有大量新闻文章或电子邮件数据集），那么某些值会更大。由于语言本身的结构，其他人也很常见。因此，例如“ is”，“ the”，“ an”，代词，语法结构等词可能会歪曲我们的矩阵并影响我们的分析。

有两种方法可以减轻这种情况。一种方法是使用“ stop_words”参数并将其值设置为“ english”。这将自动忽略在scikit-learn的内置英语停用词列表中找到的所有词（来自我们的输入文本）。

缓解此问题的另一种方法是使用[tfidf](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)方法。此方法超出了本课程的范围。

### 步骤3.1：培训和测试集 ###

既然我们已经了解了如何处理“单词袋”问题，我们可以返回到数据集并继续进行分析。 我们在这方面的第一步是将我们的数据集分为训练和测试集，以便以后可以测试模型。


> **说明：**
使用sklearn中的train_test_split方法将数据集分为训练和测试集。 分割数据
使用以下变量：
* `X_train`是我们针对`sms_message`列的训练数据。
* `y_train`是我们针对`label`列的训练数据
* `X_test`是我们针对`sms_message`列的测试数据。
* `y_test`是我们针对`label`列的测试数据

> 打印出每个培训和测试数据中的行数。


In [13]:
'''
Solution

NOTE: sklearn.cross_validation will be deprecated soon to sklearn.model_selection 
'''
# split into training and testing sets
# USE from sklearn.model_selection import train_test_split to avoid seeing deprecation warning.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


### 步骤3.2：将词袋处理应用于我们的数据集

既然我们已经分割了数据，我们的下一个目标是遵循第2步：词袋中的步骤，并将我们的数据转换为所需的矩阵格式。 为此，我们将像以前一样使用CountVectorizer()。 这里有两个步骤需要考虑：

* 首先，我们必须将训练数据（`X_train`）放入`CountVectorizer()`中并返回矩阵。
* 其次，我们必须转换测试数据（`X_test`）以返回矩阵。

请注意，`X_train`是我们数据集中`sms_message`列的训练数据，我们将使用它来训练模型。

`X_test`是我们针对`sms_message`列的测试数据，这是我们将用来（在转换为矩阵之后）进行预测的数据。 然后，我们将在以后的步骤中将这些预测与y_test进行比较。

现在，我们已经为您提供了进行矩阵转换的代码！

[实践节点]

该段的代码分为2部分。 首先，我们正在为训练数据学习词汇词典
然后将数据转换成文档术语矩阵； 其次，对于测试数据，我们只是
将数据转换为文档术语矩阵。

这类似于我们在步骤2.3中遵循的过程

我们将在变量`training_data`和`testing_data`中为学生提供转换后的数据。

In [14]:
'''
Solution
'''
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

### 步骤4.1：从头开始执行贝叶斯定理

现在我们已经有了所需格式的数据集，我们可以继续执行任务的下一部分，该算法是我们用来进行预测以将邮件分类为垃圾邮件还是非垃圾邮件的算法。请记住，在任务开始时，我们简要讨论了贝叶斯定理，但现在我们将更详细地讨论。用外行术语来说，贝叶斯定理基于与所讨论事件相关的某些其他概率来计算事件发生的概率。它由一个先验（我们知道或提供给我们的概率）和后验（我们希望使用先验来计算的概率）组成。

让我们使用一个简单的示例从头开始实现贝叶斯定理。假设我们正在尝试寻找某人患有糖尿病的可能性，因为他或她经过了测试并获得了积极的结果。
在医学领域，这类概率通常起着生死攸关的作用，因此起着非常重要的作用。

我们假设以下内容：

`P(D)`是一个人患有糖尿病的概率。它的值为`0.01`，换句话说，占总人口的1％为糖尿病（免责声明：这些值是假设，并不反映任何医学研究）。

`P(Pos)`是获得阳性测试结果的概率。

`P(Neg)`是得到阴性测试结果的概率。

`P(Pos | D)`是在假设您患有糖尿病的情况下通过检测糖尿病而获得阳性结果的可能性。其值为`0.9`。换句话说，测试在90％的时间内都是正确的。这也称为灵敏度或真正率。

`P(Neg |〜D)`是在您没有患糖尿病的情况下进行的检测糖尿病的阴性结果的可能性。该值也为`0.9`，因此90％的时间是正确的。这也称为特异性或真阴性率。

贝叶斯公式如下：

<img src ="images/bayes_formula.png" height ="242" width ="242">

* `P(A)`是A独立发生的先验概率。在我们的示例中，这是`P(D)`。这个价值是给我们的。

* `P(B)`是B独立发生的先验概率。在我们的示例中，这是`P(Pos)`。

* `P(A | B)`是给定B时A发生的后验概率。在我们的示例中，这是`P(D | Pos)`。也就是说，**某人患有糖尿病的概率，因为该人的测试结果为阳性。这是我们希望计算的值。**

* `P(B | A)`是在给定A的情况下B发生的可能性。在我们的示例中，这是`P(Pos | D)`。这个价值是给我们的。

将我们的值放入贝叶斯定理的公式中，我们得到：

`P(D|Pos) = P(D) * P(Pos|D) / P(Pos)`

可以使用灵敏度和特异性计算出获得阳性测试结果`P(Pos)`的概率，如下所示：

`P(Pos) = [P(D) * Sensitivity] + [P(~D) * (1-Specificity))]`

> **说明：**
计算获得阳性测试结果的可能性， P(Pos)

In [16]:
'''
Solution (skeleton code will be provided)
'''
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg|~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = (p_diabetes*p_pos_diabetes)+(p_no_diabetes*(1-p_neg_no_diabetes))
print('The probability of getting a positive test result P(Pos) is: {}',format(p_pos))

The probability of getting a positive test result P(Pos) is: {} 0.10799999999999998


**使用所有这些信息，我们可以如下计算后验：**
    
假设某人患有糖尿病的概率为阳性，则该测试结果为：

`P(D|Pos) = (P(D) * Sensitivity)) / P(Pos)`

考虑到个人没有糖尿病的概率，则该人获得了阳性测试结果：

`P(~D|Pos) = (P(~D) * (1-Specificity)) / P(Pos)`

我们后代的总和将始终等于1。

> **说明：**
给定某人患有糖尿病的可能性，计算该人的概率。
换句话说，计算P(D | Pos)。

> 公式为：`P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)`

In [18]:
'''
Solution
'''
# P(D|Pos)
p_diabetes_pos = (p_diabetes*p_pos_diabetes)/(p_pos)
print('Probability of an individual having diabetes, given that that individual got a positive test result is:\
',format(p_diabetes_pos)) 

Probability of an individual having diabetes, given that that individual got a positive test result is: 0.08333333333333336


**说明：**

考虑到该个体获得阳性测试结果，计算该个体未患有糖尿病的概率。
换句话说，计算P(~D|Pos)。

公式为: `P(~D|Pos) = P(~D) * P(Pos|~D) / P(Pos)`

注意， `P(Pos|~D) `可以计算为`1 - P(Neg|~D)`。

因此:
`P(Pos|~D) = p_pos_no_diabetes = 1 - 0.9 = 0.1`

In [20]:
'''
Solution
'''
# P(Pos|~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos = (p_no_diabetes * p_pos_no_diabetes) / p_pos
print ('Probability of an individual not having diabetes, given that that individual got a positive test result is:'\
,p_no_diabetes_pos)

Probability of an individual not having diabetes, given that that individual got a positive test result is: 0.9166666666666669


恭喜你！ 您从头开始实现了贝叶斯定理。 您的分析表明，即使您得到阳性检测结果，您实际患糖尿病的机率也只有8.3％，而没有糖尿病的机率只有91.67％。 当然，这是假设整个人口中只有1％患有糖尿病，这当然只是一个假设。

**“朴素贝叶斯”中的“朴素”一词是什么意思?**

朴素贝叶斯中的“朴素”一词源于以下事实：该算法将其用于做出预测的功能考虑为彼此独立的事实，这种情况可能并非总是如此。 因此，在我们的糖尿病示例中，我们仅考虑一项功能，即测试结果。 假设我们添加了另一个功能“锻炼”。 假设此功能的二进制值为“ 0”和“ 1”，其中前者表示个人每周练习少于或等于2天，而后者则表示个人每天练习大于或等于3天 周。 如果我们必须使用测试结果和“运动”功能的值这两个功能来计算最终概率，贝叶斯定理将失败。 朴素贝叶斯（Naive Bayes）是贝叶斯（Bayes）定理的扩展，它假定所有特征彼此独立。

### 步骤4.2：从零开始实施朴素贝叶斯

既然您已经了解了贝叶斯定理的来龙去脉，那么我们将其扩展为考虑具有更多特征的情况。

假设我们有两个政党候选人，即绿党的“吉尔·斯坦”和自由党的“加里·约翰逊”，我们每个候选人都有说“自由”，“移民”和“ 演讲时的“环境”：

* Probability that Jill Stein says 'freedom': 0.1 ---------> `P(F|J)`
* Probability that Jill Stein says 'immigration': 0.1 -----> `P(I|J)`
* Probability that Jill Stein says 'environment': 0.8 -----> `P(E|J)`


* Probability that Gary Johnson says 'freedom': 0.7 -------> `P(F|G)`
* Probability that Gary Johnson says 'immigration': 0.2 ---> `P(I|G)`
* Probability that Gary Johnson says 'environment': 0.1 ---> `P(E|G)`

并且我们还假设吉尔·斯坦因发表演讲的概率`P(J)` 为`0.5`，而加里·约翰逊的概率为`P(G) = 0.5`.。

鉴于此，如果我们不得不找到吉尔·斯坦因说“自由”和“移民”这两个词的可能性怎么办？ 这就是朴素贝叶斯定理发挥作用的地方，因为我们正在考虑“自由”和“移民”两个特征。

现在我们可以定义朴素贝叶斯定理的公式：

<img src="images/naivebayes.png" height="342" width="342">

在这里，`y`是类变量，在本例中为候选者的名称，在`x1`至`xn`中为特征向量，在本例中为单个单词。 该定理假设每个特征向量或单词（`xi`）彼此独立。

为了解决这个问题，我们必须计算以下后验概率：

* `P(J|F,I)`：吉尔·斯坦（Jill Stein）说“自由与移民”的可能性。

    使用公式和我们对贝叶斯定理的了解，我们可以如下计算：`P(J|F,I) = (P(J) * P(F|J) * P(I|J)) / P(F,I)`。 这里的`P(F,I)` 是讲话中说“自由”和“移民”两个词的可能性。
    

* `P(G|F,I)`：加里·约翰逊（Gary Johnson）说自由与移民一词的可能性。
    
     使用公式，我们可以如下计算：`P(G|F,I) = (P(G) * P(F|G) * P(I|G)) / P(F,I)`

**说明**：计算演讲中说“自由”和“移民”两个词的可能性，或者
P(F，I)。

第一步是将吉尔·斯坦（Jill Stein）与个人讲话的概率相乘
说“自由”和“移民”一词的可能性。 将此存储在名为p_j_text的变量中

第二步是加里·约翰逊（Gary Johnson）与他的个人讲话的可能性相乘
说“自由”和“移民”一词的可能性。 将此存储在名为p_g_text的变量中

第三步是将这两个概率加在一起，您将获得P(F，I)。

In [22]:
'''
Solution: Step 1
'''
# P(J)
p_j = 0.5

# P(F/J)
p_j_f = 0.1

# P(I/J)
p_j_i = 0.1

p_j_text = p_j*p_j_f*p_j_i
print(p_j_text)

0.005000000000000001


In [23]:
'''
Solution: Step 2
'''
# P(G)
p_g = 0.5

# P(F/G)
p_g_f = 0.7

# P(I/G)
p_g_i = 0.2

p_g_text = p_g * p_g_f * p_g_i
print(p_g_text)

0.06999999999999999


In [24]:
'''
Solution: Step 3: Compute P(F,I) and store in p_f_i
'''
p_f_i = p_j_text + p_g_text
print('Probability of words freedom and immigration being said are: ', format(p_f_i))

Probability of words freedom and immigration being said are:  0.075


现在我们可以计算出 `P(J|F,I)`的概率，即吉尔·斯坦因说“自由与移民”的概率，以及 `P(G|F,I)`,的概率，即加里·约翰逊的概率 说“自由与移民”一词。

**说明：**
使用公式P（J | F，I）=（P（J）* P（F | J）* P（I | J））/ P（F，I）计算P（J | F，I）并存储 它在变量p_j_fi中

In [25]:
'''
Solution
'''
p_j_fi = (p_j*p_j_i*p_j_f)/(p_f_i)
print('The probability of Jill Stein saying the words Freedom and Immigration: ', format(p_j_fi))

The probability of Jill Stein saying the words Freedom and Immigration:  0.06666666666666668


**说明：**
使用公式P（G | F，I）=（P（G）* P（F | G）* P（I | G））/ P（F，I）计算P（G | F，I）并存储 它在变量p_g_fi中

In [26]:
'''
Solution
'''
p_g_fi = (p_g*p_g_i*p_g_f)/(p_f_i)
print('The probability of Gary Johnson saying the words Freedom and Immigration: ', format(p_g_fi))

The probability of Gary Johnson saying the words Freedom and Immigration:  0.9333333333333332


正如我们所看到的，就像在贝叶斯定理案例中一样，我们后代的总和等于1。恭喜！ 您从头开始实现了朴素贝叶斯定理。 我们的分析表明，绿党的吉尔·斯坦因在演讲中使用“自由”和“移民”的可能性只有6.6％，而自由党的加里·约翰逊则只有93.3％。

朴素贝叶斯的另一个更一般的例子是当我们在搜索引擎中搜索“萨克拉曼多国王”一词时。为了使我们获得与Scramento Kings NBA篮球队有关的结果，搜索引擎需要能够将两个单词关联在一起，而不是单独对待它们，在这种情况下，我们将获得标有“ Sacramento”标签的图像的结果。例如城市景观的图片和“国王”的图片，当我们想要获取的是篮球队的图片时，这些图片可能是历史上的王冠或国王的图片。这是搜索引擎将单词视为独立实体并因此在其方法上“天真”的经典案例。


将其应用于我们将邮件归类为垃圾邮件的问题后，朴素贝叶斯算法*单独查看每个单词，而不将其视为关联的实体*，它们之间没有任何形式的链接。对于垃圾邮件检测器，这通常可以工作，因为某些危险信号词几乎可以保证将其分类为垃圾邮件，例如带有“ viagra”之类的单词的电子邮件通常被归类为垃圾邮件。

### 步骤5：使用scikit-learn的朴素贝叶斯实现

值得庆幸的是，sklearn有几个可以使用的朴素贝叶斯实现，因此我们不必从头开始。 我们将使用sklearns的sklearn.naive_bayes方法对数据集进行预测。

具体来说，我们将使用多项朴素的贝叶斯实现。 此特定分类器适用于具有离散功能的分类（例如，在我们的案例中，为文本分类使用单词计数）。 它以整数字数作为输入。 另一方面，由于高斯朴素贝叶斯假设输入数据具有高斯（正态）分布，因此它更适合于连续数据。

In [ ]:
'''
Instructions:

We have loaded the training data into the variable 'training_data' and the testing data into the 
variable 'testing_data'.

Import the MultinomialNB classifier and fit the training data into the classifier using fit(). Name your classifier
'naive_bayes'. You will be training the classifier using 'training_data' and y_train' from our split earlier. 
'''

In [28]:
'''
Solution
'''
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
'''
Instructions:
Now that our algorithm has been trained using the training data set we can now make some predictions on the test data
stored in 'testing_data' using predict(). Save your predictions into the 'predictions' variable.
'''

In [29]:
'''
Solution
'''
predictions = naive_bayes.predict(testing_data)

Now that predictions have been made on our test set, we need to check the accuracy of our predictions.

### 步骤6：评估我们的模型

现在我们已经对测试集进行了预测，我们的下一个目标是评估模型的运行情况。这样做的机制多种多样，但首先让我们快速回顾一下它们。

**准确性**衡量分类器做出正确预测的频率。它是正确预测的数量与预测的总数（测试数据点的数量）之比。

**精确度**告诉我们，我们归类为垃圾邮件的邮件比例实际上是垃圾邮件。它是真实肯定（分类为垃圾邮件的单词，实际上是垃圾邮件）与所有肯定（所有分类为垃圾邮件的单词，与分类是否正确无关）的比率，即`[True Positives/(True Positives + False Positives)]`。

**召回（敏感度）**告诉我们，实际上是垃圾邮件的邮件中，有多少比例被我们归类为垃圾邮件。它是真实肯定（分类为垃圾邮件的单词，实际上是垃圾邮件）与所有实际上是垃圾邮件的单词的比率，换句话说，是`[True Positives/(True Positives + False Negatives)]`。

对于像我们这种情况那样在分类分布中出现偏差的分类问题，例如，如果我们有100条短信，而只有2条是垃圾邮件，而其余98条不是，则准确性本身并不是一个很好的指标。我们可以将90封邮件归类为非垃圾邮件（包括2封属于垃圾邮件，但我们将其归类为非垃圾邮件，因此它们将是误报），将10封邮件归类为垃圾邮件（所有10封误报邮件），但仍可获得相当不错的准确性得分。在这种情况下，精确度和召回率非常有用。可以将这两个指标结合起来以获得F1分数，该分数是精度和召回分数的加权平均值。该分数的范围是0到1，其中1是最佳的F1分数。

我们将使用所有四个指标来确保我们的模型运行良好。对于所有4个指标，其值的范围可以从0到1，得分都尽可能接近1，可以很好地指示我们的模型的运行状况。

> **说明：**
使用测试数据y_test和预测来计算模型的准确性，精度，召回率和F1分数
您之前所做的存储在“预测”变量中。

In [31]:
'''
Solution
'''
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562


### 步骤7：结论

与其他分类算法相比，朴素贝叶斯（Naive Bayes）具有的主要优点之一是其处理大量特征的能力。 在我们的例子中，每个单词都被当作一个特征，并且有成千上万个不同的单词。 而且，即使存在不相关的功能，它的性能也很好，并且相对不受它们的影响。 它的另一个主要优点是相对简单。 朴素贝叶斯（Naive Bayes）的开箱即用效果很好，几乎不需要调整其参数，除非通常在已知数据分布的情况下。

它很少过拟合数据。 另一个重要的优点是，对于可以处理的数据量，其模型训练和预测时间非常快。 总而言之，朴素贝叶斯真的是算法的瑰宝！

恭喜你！ 您已经成功设计了一个模型，可以有效地预测SMS消息是否为垃圾邮件！

感谢您与我们一起学习！